In [ ]:
8. Broadcast Join
Description: A broadcast join is a performance optimization where the smaller DataFrame is broadcasted to all the worker nodes, allowing for a much faster join when working with large datasets. Spark automatically uses broadcast join if it detects that the DataFrame size is below a threshold, but you can force it.
Use Case: When one DataFrame is much smaller than the other, and you want to avoid shuffling the larger DataFrame.


from pyspark.sql.functions import broadcast

df1.join(broadcast(df2), df1['id'] == df2['id']).show()


In [ ]:
#Optimizing Joins in PySpark
'''
 * Partitioning: Ensure both DataFrames are appropriately partitioned to avoid shuffling large amounts of data.
 * Broadcast Join: Use it for joining large datasets with small ones.
 * Skew Handling: If one of your DataFrames has skewed data (i.e., some keys appear much more often than others), this can lead to 
performance bottlenecks. Use salting techniques to handle skewed joins.
'''

In [ ]:
#In **PySpark**, joins are performed internally using different strategies, based on the size of the datasets being joined and the type of 
#join operation. Heres a breakdown of how joins are processed and the underlying mechanics:

### 1. **Shuffling and Data Exchange**
'''
   When PySpark performs a join between two large datasets, it needs to bring matching keys together. This often requires **shuffling**, 
   which is the process of redistributing data across the cluster. The rows with the same key from different partitions and nodes are 
   brought to the same node for the join to be performed.

   - **Shuffling** involves:
     - Partitioning the data by key.
     - Moving data across the network between different nodes.
     - Sorting the data to prepare for the join.

   Shuffling can be very expensive, especially with large datasets, because it involves reading and writing intermediate data to disk 
   and transferring it across the network.
'''

In [ ]:
### 2. **Join Algorithms**
'''
   PySpark uses different algorithms to execute the join, depending on the characteristics of the datasets and the type of join:
'''
#### 2.1. **Sort-Merge Join**
'''
   - **Description**: This is the default algorithm for large joins in PySpark. It is used when both datasets are too large to fit in 
   memory and need to be shuffled and sorted before joining.
   - **Steps**:
     1. **Shuffle** the data from both datasets so that rows with the same key end up on the same partition.
     2. **Sort** the data in each partition by the join key.
     3. **Merge** the sorted partitions by comparing keys and producing the joined output.

   - **Best for**: Large datasets that are already partitioned or sorted by key.

   - **Example Scenario**:
     If two large DataFrames are joined on a column `id`, the records are shuffled and sorted based on `id` in each partition, and 
     then a merge process is executed to match records.

   - **Performance Impact**: While sort-merge join is efficient for large datasets, it incurs a high cost due to the shuffle and sort phases.
'''

#### 2.2. **Broadcast Join**
'''
   - **Description**: A **broadcast join** occurs when one dataset is much smaller than the other. The smaller dataset is broadcasted to all nodes, allowing the larger dataset to be joined without shuffling. This prevents the overhead of a full shuffle for the larger dataset.
   - **Steps**:
     1. PySpark copies the smaller dataset (broadcast) to every node in the cluster.
     2. The larger dataset is scanned, and for each row, the corresponding row in the smaller (broadcasted) dataset is fetched and joined.

   - **Best for**: Joins where one of the DataFrames is small enough to fit into memory of all nodes.

   - **Example Scenario**:
     If you have a large sales dataset and a small lookup table (e.g., mapping country codes to country names), the lookup table can be broadcasted to avoid shuffling the large dataset.

   - **Performance Impact**: **Broadcast joins** are very fast for small datasets but should only be used when the broadcasted DataFrame fits in memory.

   - **Note**: PySpark automatically performs broadcast joins when it detects that the smaller dataset is below a certain size threshold, but you can manually force it using the `broadcast()` function.
'''

#### 2.3. **Shuffle Hash Join**
'''
   - **Description**: In this join, the smaller dataset is used to build a hash map, and the larger dataset is scanned and matched against the hash map. It requires a shuffle of the larger dataset but avoids a full sort, which can be more efficient in some cases.
   - **Steps**:
     1. **Shuffle** the data based on the join key to ensure that rows with matching keys are in the same partition.
     2. Build a **hash map** from the smaller dataset.
     3. For each row in the larger dataset, check the hash map for a matching key.
  
   - **Best for**: Situations where one dataset is much smaller but not small enough for broadcast.

   - **Performance Impact**: Hash joins are generally faster than sort-merge joins because they avoid sorting, but they still incur a shuffle.
'''

#### 2.4. **Broadcast Hash Join**
'''
   - **Description**: A variant of the hash join where the smaller dataset is broadcasted to all nodes, and then each node builds a hash map of the broadcasted data.
   - **Steps**:
     1. **Broadcast** the smaller dataset to every node.
     2. Build a **hash map** from the smaller dataset on each node.
     3. For each row in the larger dataset, check the hash map for a matching key.

   - **Best for**: Smaller datasets where shuffling can be avoided altogether.
'''


In [ ]:

### 3. **Join Execution Plan**
'''
PySpark provides insight into the execution of a join using the `explain()` method, which shows the **logical** and **physical plan** of 
the join operation. This is crucial for understanding how PySpark will execute the join and whether any optimizations (like broadcast join) 
are being applied.

   Example:
   ```python
   df1.join(df2, 'id').explain()
   ```
   Output (example):
   ```
   == Physical Plan ==
   *(5) SortMergeJoin [id#4L], [id#9L], Inner
   :- *(2) Sort [id#4L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#4L, 200), true, [id=#37]
   +- *(4) Sort [id#9L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#9L, 200), true, [id=#38]
   ```
'''

In [ ]:
### 4. **Optimizing Joins**
#PySpark offers several ways to optimize joins:

# 1.**Broadcasting Small Tables**: For joins where one table is significantly smaller, broadcasting can reduce shuffle costs. 
# Use `broadcast()` to force broadcasting if needed.
from pyspark.sql.functions import broadcast
df1.join(broadcast(df2), 'id').show()


#2. **Partitioning**: If your data is already partitioned on the join key, PySpark will avoid an expensive shuffle. 
# Ensure that both datasets are partitioned similarly.
df1.repartition('id').join(df2.repartition('id'), 'id').show()

#3. *Bucketing**: If the datasets are large, you can bucket them by the join key for more efficient joins.
df1.write.bucketBy(10, 'id').saveAsTable('bucketed_table')

#4.  **Predicate Pushdown**: Use predicates (filters) to reduce the size of the datasets before performing the join, thereby 
# reducing the shuffle size.
df1.filter(df1['value'] > 100).join(df2, 'id').show()
    

#5. **Skewed Joins**: If one dataset has highly skewed keys (i.e., some keys occur much more often than others), 
# you can "salt" the join by adding random noise to the key to distribute the load.

### Conclusion
'''
    Internally, PySparks joins are optimized based on the size and characteristics of the data, using strategies 
    like **sort-merge join**, **hash join**, and **broadcast join**. The decision to use shuffling, sorting, or broadcasting depends on 
    factors like data size and memory constraints, and it is important to understand how PySpark handles these operations to optimize your 
    joins effectively.
'''
